# Homework - Logistic Regression

<hr style="clear:both">

This homework is part of a series of exercises for the CIVIL-226 Introduction to Machine Learning for Engineers course at EPFL. Copyright (c) 2022 [VITA](https://www.epfl.ch/labs/vita/) lab at EPFL  
Use of this source code is governed by an MIT-style license that can be found in the LICENSE file or at https://www.opensource.org/licenses/MIT

**Author(s):** [David Mizrahi](mailto:david.mizrahi@epfl.ch), [Tom Winandy](mailto:tom.winandy@epfl.ch) and [Luc Reveyron](mailto:luc.reveyron@epfl.ch)

**2022 Revision:** [Brian Sifringer](mailto:brian.sifringer@epfl.ch), [Bastien Van Delft](mailto:bastien.vandelft@epfl.ch), [Taylor Mordan](mailto:taylor.mordan@epfl.ch)

<hr style="clear:both">


### Name and SCIPER

YOUR ANSWER HERE

### Homework info
**Released:** Wednesday March 23, 2022  
**Submission**: Thursday March 31, 2022 (before 11:59PM) on Moodle  
**Grade weight:** 20% of the overall grade  
**This is an individual assignment, the exchange of code between students is forbidden.**


This homework is composed of 4 parts:
- Part 1: Logistic regression (45 pts)
- Part 2: Regularization (20 pts)
- Part 3: Cross-validation (20 pts)
- Part 4: Evaluation metrics (15 pts)

**Total:** 100 pts

**Work through this exercise in order, as functions implemented early on can be used for later parts.**

**Tips:**
- Use the Table of Contents extension to quickly navigate to each part.
- In order to ensure that there are no hidden states in your notebook, frequently restart the kernel using: **Kernel -> Restart Kernel and Run all Selected Cells...**

Good luck!

### Imports

In [ ]:
# Function to align all tables to the left (useful for later on)

In [ ]:
%%html
<style>
table {float:left}
</style>

In [ ]:
### START CODE HERE ###
...
### END CODE HERE ###

# Part 1: Logistic regression

The first part of this exercise consists of implementing logistic regression for a binary classification problem. In this part, you will use a slightly modified version of the Palmer Penguins dataset.

## 1.1. Dataset

### Loading the data & normalization

First, you'll work on a relatively simple dataset: a modified version of the Palmers Penguins dataset with only two features (bill length and flipper length), and two species (Gentoo and Chinstrap). Using `helpers.preprocess_data()` (check helpers.py for more info), we'll obtain a training and test set.

In [ ]:
penguins = pd.read_csv('data/penguins.csv')
penguins.head(5)

In [ ]:
X_train_penguins, y_train_penguins, X_test_penguins, y_test_penguins, feature_names, label_map = helpers.preprocess_data(penguins, label="species", train_size=0.70, seed=42)

Next, you need to normalize this data with the mean and standard deviation from the training set and keep the original variable name. For example, the normalized `X_train_penguins` should be called `X_train_penguins`.

**Task: Normalize the training and test data using `helpers.normalize()`**

**Note:** Remember that you should not use any of the knowledge you get from the test data when implementing a model. This includes the normalization step, where you should use the mean and standard deviation of the **training set** to normalize both the **training** and **test** set.

In [ ]:
# Normalize features of the training and test set using the mean and std of the training set features
### START CODE HERE ###
...
### END CODE HERE ###

In [ ]:
# These assertions check that your normalization didn't go completely wrong. 
# Passing these assertions does not mean you will automatically get all points for that question
# We may use other tests to check the correctness of your implementation
assert np.allclose(X_train_penguins.mean(axis=0), 0)
assert np.allclose(X_train_penguins.std(axis=0), 1)

Let's preview the arrays for the training and test set.

In [ ]:
# Preview of X_train_penguins and y_train_penguins (separation of the features and the labels)
print('Training set features:')
print(f'X_train_penguins: \n {X_train_penguins[:3]}')

print('\nTraining set labels:')
print(f'y_train_penguins: \n {y_train_penguins[:3]}')

**Expected output:**

|                 |                                                  |
|-----------------|--------------------------------------------------|
| **X_train_penguins** | [[ 0.75973314  1.26418483]<br>[ 0.62957146 -0.72722878]<br>[-1.84350033 -1.87612124]] |
| **y_train_penguins**  | [1 0 0]


In [ ]:
# Preview of X_train_penguins and y_train_penguins (separation of the features and the labels)
print('Test set features:')
print(f'X_test_penguins: \n {X_test_penguins[:3]}')

print('\nTraining set labels:')
print(f'y_test_penguins: \n {y_test_penguins[:3]}')

**Expected output:**

|                 |                                                  |
|-----------------|--------------------------------------------------|
| **X_test_penguins** |  [[-1.58317698  0.1918852 ]<br>[-1.48555573  1.11099917]<br>[ 0.59703104  0.57484935]] |
| **y_test_penguins**  | [1 1 1]


In [ ]:
# Show shapes
print('Training set shape:')
print(f'X: {X_train_penguins.shape}, y: {y_train_penguins.shape}')

print('\nTest set shape:')
print(f'X: {X_test_penguins.shape}, y: {y_test_penguins.shape}')

### Notation

Now that we have pre-processed our dataset, here's how it looks:

- features: $\boldsymbol{X} \in \mathbb{R}^{N \times D}$, $\forall \ \boldsymbol{x}^{(i)} \in \boldsymbol{X}: \boldsymbol{x}^{(i)} \in \mathbb{R}^{D}$
- labels: $\boldsymbol{y} \in \mathbb{R}^{N}$, $\forall \ y^{(i)} \in \boldsymbol{y}: y^{(i)} \in \{0, 1\}$ 
  
 where $N$ is the number of examples in our dataset, and $D$ is the number of features per example  
 

For the weights, we have:
 
 
 - weights: $\mathbf{w} \in \mathbb{R}^{D}$
 - bias: $b \in \mathbb{R}$

 **Note:**
 
 $\boldsymbol{X}$ is called the design matrix, where $\boldsymbol{X}_{i, :}$ denotes $\boldsymbol{x}^{(i)}$.  
 Note that a single example $\boldsymbol{x}^{(i)}$ is a column vector of shape $(D \times 1)$, while the design matrix $\boldsymbol{X}$ is of shape $(N \times D)$, where each row represents an example and each column represents a feature.

## 1.2. Diving into logistic regression

In this section, you are going to implement the different functions needed for logistic regression.

### Sigmoid

A key element of logistic regression is the sigmoid function. This function takes any real-valued input and outputs a value in [0, 1]. The sigmoid function is defined as:
$$\sigma(z)= \frac {e^z}{1+e^z}= \frac{1}{1+e^{-z}}$$

Here's a plot of the sigmoid:

<img src="images/sigmoid.png" alt="Sigmoid" style="width: 450px"/>

**Task: Implement `sigmoid()`**  

**Hint:** Use `np.exp(x)` to take the exponential of a number. You can find the documentation for this method [here](https://numpy.org/doc/stable/reference/generated/numpy.exp.html).

In [ ]:
def sigmoid(z: np.ndarray) -> np.ndarray:
    """ Sigmoid function
    
    Args:
        z: Input data of shape (N, )
        
    Returns:
        np.ndarray: Sigmoid of z of shape (N, ), where each value is in [0, 1].
    """
    ### START CODE HERE ###
    ...
    ### END CODE HERE ###
    return s

In [ ]:
# Verify implementation
a = sigmoid(np.array([3, 0.5, -1]))
print(f'a: {np.round(a, 4)}')

**Expected output:**

|   |                                                  |
|---|--------------------------------------------------|
| **a** | [0.9526 0.6225 0.2689] |

### Logistic output

Now that you have a function that computes the sigmoid function, you can implement a function that gives the logistic output. As a reminder, this function outputs the estimated probability that y = 1 for an input $\boldsymbol{x}$, that is: $ \hat{y}^{(i)} = p(y^{(i)} = 1  | \boldsymbol{x}^{(i)})$.  
The logistic output (of a single example) is defined as:
$$\hat{y}^{(i)} = \sigma(\mathbf{w}^{T} \boldsymbol{x}^{(i)} + b) = \frac{1}{1 + e^{-(\mathbf{w}^{T}  \boldsymbol{x}^{(i)} + b)}}$$ 

For all examples, the output is defined as: 
$$\mathbf{\hat{y}} = \sigma(\mathbf{X} \mathbf{w} + b \mathbf{1})$$
where $\mathbf{1}$ is a vector of ones of shape $(N \times 1)$, and the sigmoid is applied element-wise.

**Task: Implement `logistic_output()`**

**Hint:** To vectorize this operation , use `np.matmul(a, b)` (or equivalently `a @ b`) for matrix multiplication.  
You can find the documentation for this method [here](https://numpy.org/doc/stable/reference/generated/numpy.matmul.html).

**Hint:** Remember that you have already coded the sigmoid function. This might help you in the logistic output implementation. 

In [ ]:
def logistic_output(X: np.ndarray, w: np.ndarray, b: float) -> np.ndarray:
    """ Output of logistic regression
    
    Args:
        X: Dataset of shape (N, D)
        w: Weights of logistic regression model of shape (D, )
        b: bias, a scalar
    Returns:
        y_hat (np.ndarray): Output of logistic regression of shape (N, )
    """
    ### START CODE HERE ###
    ...
    ### END CODE HERE ###
    return y_hat

In [ ]:
# Verify implementation
X = X_train_penguins[:3]
w = np.array([1, 1])
b = 1

y_hat = logistic_output(X, w, b)
print(f'y_hat: {np.round(y_hat, decimals=4)}')

**Expected output:**

|                 |                                                  |
|-----------------|--------------------------------------------------|
| **y_hat** | [0.9536 0.7114 0.0618] |

### Binary cross-entropy loss

In order to train your model, you also need a loss function, which penalizes outputs based on how far off they are from the ground-truth. Here, we'll use the logistic loss / binary cross-entropy loss.

It is defined as:

$$\text{BCE}(\mathbf{w}, b) = - \frac{1}{N}\sum^{N}_{i=1} y^{(i)} \log(\hat{y}^{(i)}) + (1-y^{(i)}) \log(1- \hat{y}^{(i)})$$ where $\log(x)$ refers to the natural logarithm of $x$.

**Task: Implement `bce_loss()`**

**Hint:** Use `np.log(x)` to take the natural logarithm of x. You can find the documentation for this method [here](https://numpy.org/doc/stable/reference/generated/numpy.log.html).

<div class="alert alert-info">
Due to floating-point arithmetic, values very close to 0 will get rounded to 0 (likewise for values very close to 1). 

However, the natural logarithm log(x) is undefined for values equal to 0. An easy way to fix this is to add a small term $\epsilon$ to the logarithm. For example, write `np.log(x + epsilon)` instead of `np.log(x)`. 
    
In order to get full points (and avoid problems in later parts of this homework), add  `epsilon` (set to $10^{-9}$) whenever you call `np.log()`.
</div>

In [ ]:
def bce_loss(X: np.ndarray,  y: np.ndarray, w: np.ndarray, b: float) -> float:
    """ Binary cross-entropy loss function
    
    Args:
        X: Dataset of shape (N, D)
        y: Labels of shape (N, ).
        w: Weights of logistic regression model of shape (D, )
        b: bias, a scalar
    
    Returns:
        float: binary cross-entropy loss.
    """
    # Add the epsilon term to the np.log() in your implementation (e.g. do np.log(x + epsilon) instead of np.log(x))
    # Epsilon is there to avoid log(0)
    epsilon = 1e-9
    
    ### START CODE HERE ###
    ...
    ### END CODE HERE ###
    
    return loss

In [ ]:
# Verify implementation
X = X_train_penguins[:3]
y  = y_train_penguins[:3]
w = np.array([1, 1])
b = 1

loss = bce_loss(X, y, w, b)
print(f'Loss: {loss:.3f}')

**Expected output:**

|                 |                                                  |
|-----------------|--------------------------------------------------|
| **Loss** | 0.451|

### Binary cross-entropy loss gradient

After having computed the loss, you need to update the parameters of the model in order to decrease it. This can be done with gradient descent, and for that, you'll need the gradient of your loss function.

In class, we have seen the gradient of the binary cross-entropy loss defined as:
$$\frac{\partial \text{BCE}(\mathbf{w},b)}{\partial \mathbf{w}} = \frac{1}{N}\sum^{N}_{i=1}\boldsymbol{x}^{(i)} (\hat{y}^{(i)} - y^{(i)}) $$
$$ \frac{\partial \text{BCE}(\mathbf{w},b)}{\partial b} = \frac{1}{N}\sum^{N}_{i=1}\hat{y}^{(i)} - y^{(i)}$$

A vectorized implementation of that first expression is the following:  

$$\frac{\partial \text{BCE}(\mathbf{w},b)}{\partial \mathbf{w}} = \frac{1}{N} \ \mathbf{X}^T (\mathbf{\hat{y}} - \mathbf{y})$$

**Task: Implement `bce_gradient()`**

In [ ]:
def bce_gradient(X: np.ndarray,  y: np.ndarray, w: np.ndarray, b: float) -> Tuple[np.ndarray, float]:
    """ Gradient of the binary-cross entropy loss
    
    Args:
        X: Dataset of shape (N, D)
        y: Labels of shape (N, )
        w: Weights of logistic regression model of shape (D, )
        b: bias, a scalar
        
    Returns:
        dw (np.ndarray) gradient of the loss with respect to w of shape (D, )
        db (float) gradient of the loss with respect to b, a scalar
    """
    ### START CODE HERE ###
    ...
    ### END CODE HERE ###
    
    return dw, db
    

In [ ]:
# Verify implementation
X = X_train_penguins[:3]
y  = y_train_penguins[:3]
w = np.array([1, -1])
b = -0.5

dw, db = bce_gradient(X, y, w, b)
print(f'dw: {np.round(dw, decimals=4)}')
print(f'db: {db:.4f}')


**Expected output:**

|    |                                   |
|----|-----------------------------------|
| **dw** | [-0.2748 -0.7195] |
| **db** | 0.1184                           |

### Classification

The output of logistic regression estimates $ \hat{y}^{(i)} = P(y^{(i)} = 1  | \boldsymbol{x}^{(i)})$.

Based on this output, you'll want to classify each example in one of the 2 categories. To do so, you'll need to set $\text{predicted label}^{(i)}$ to 1 when $ \hat{y}^{(i)} \gt 0.5$ (as it means it is more likely to be of class 1 than of class 0), and to 0 otherwise. 

**Task: Implement `classify()`**

**Hint:** Use `np.where(condition, [x, y])`. You can find the documentation for this method [here](https://numpy.org/doc/stable/reference/generated/numpy.where.html).

In [ ]:
def classify(y_hat: np.ndarray) -> np.ndarray:
    """ Classification function for binary class logistic regression. 
    
    Args:
        y_hat (np.array): Output of logistic regression of shape (N, ).
    Returns:
        np.array: Label assignments of data of shape (N, )
    """
    ### START CODE HERE ###
    ...
    ### END CODE HERE ###
    return labels_pred

In [ ]:
y_hat = np.array([0.25, 0.75])
print(classify(y_hat))

**Expected output:** [0 1] 

### Accuracy

To measure how well your model is doing, we'll first consider the accuracy as our metric. It corresponds to the fraction of predictions our model got right.

$$\text{Accuracy} = \frac{\text{Number of correct predictions}}{\text{Total number of predictions}}$$



**Task: Implement `accuracy()`**

In [ ]:
def accuracy(labels_gt: np.ndarray, labels_pred: np.ndarray) -> float:
    """Computes the accuracy.

    Args:
        labels_gt: labels (ground-truth) of shape (M, ).
        labels_pred: Predicted labels of shape (M, ).

    Returns:
        float: Accuracy, in range [0, 1].
    """
    ### START CODE HERE ###
    ...
    ### END CODE HERE ###

In [ ]:
# Check that output is in [0, 1] (ensures that output is not a percentage)
assert 0.0 <= accuracy(np.array([1, 0]), np.array([1, 1])) <= 1.0


## 1.3. Training
You will now implement the training process by putting together all the functions you have implemented so far.

Here are the different steps of the training process:
- Compute the output of logistic regression (`y_hat`)
- Compute the loss of the model (`loss`)
- Compute the derivates w.r.t to the weights `dw` and the bias `db`
- Update `w` and `b`

Recall that a gradient step is: 
$$ \mathbf{w}  := \mathbf{w} - \alpha \frac{\partial J}{\partial \mathbf{w}} $$


$$b  := b - \alpha \frac{\partial J}{\partial b} $$ where $J$ is the loss function and $\alpha$ is the learning rate.

**Task: Complete the following `train_logistic_regression()` function.**

In [ ]:
def train_logistic_regression(X: np.ndarray, 
                              y: np.ndarray, 
                              max_iters: int = 101, 
                              lr: float = 0.5, 
                              loss_freq: int = 0) -> Tuple[np.ndarray, float, dict]:
    """ Training function for binary class logistic regression using gradient descent
    
    Args:
        X: Dataset of shape (N, D).
        y: Labels of shape (N, ).
        max_iters: Maximum number of iterations. Default : 100
        lr: The learning rate of  the gradient step. Default : 1
        loss_freq : Prints the loss every `loss_freq` iterations. Default : 0
        
    Returns:
        w: weights of shape (D, )
        b: scalar
        viz_d: dict used for visualizations
    """
    
    # Initialize weights
    np.random.seed(0)
    w = np.random.normal(0, 1, size=(X.shape[1], ))
    b = 0
    
    # Initialize dict with lists to keep track of loss, accuracy, weight and bias evolution
    logger = {'loss': [], 
             'acc': [], 
             'w': [],
             'b': []
            }
    
    
    for i in range(max_iters):
        # Compute loss, dw, db and update w and b 
        ### START CODE HERE ###
        ...
        ### END CODE HERE ###
        
        # Keep track of parameter, loss and accuracy values for each iteration
        logger['w'].append(w)
        logger['b'].append(b)
        logger['loss'].append(loss)
        y_hat = logistic_output(X, w, b)
        logger['acc'].append(accuracy(y, classify(y_hat)))
        
        if (loss_freq !=0) and i % loss_freq == 0:
            print(f'Loss at iter {i}: {loss:.5f}')
        
    if (loss_freq != 0):
        print('\nFinal loss: {:.5f}'.format(logger['loss'][-1]))
        
    return w, b, logger

Now, run the following cell with the default hyperparameters (max_iters=101, lr=0.5) to train your model parameters (`w` & `b`).

In [ ]:
w, b, logger = train_logistic_regression(X_train_penguins, y_train_penguins, max_iters=101, lr=0.5, loss_freq=5)

**Expected output:**

|            |         |
|------------|---------|
| **Final loss**  | 0.10350 |

## 1.4. Results

### Model accuracy
Great, your model is trained! But how well does it perform? To find out, run the following cells to get the accuracy of your trained model on the train and test set.

In [ ]:
# Train acc
y_hat = logistic_output(X_train_penguins, w, b)
acc = accuracy(y_train_penguins, classify(y_hat))
print(f'Train accuracy: {100 * acc:.2f}%')

In [ ]:
# Test acc
y_hat = logistic_output(X_test_penguins, w, b)
acc = accuracy(y_test_penguins, classify(y_hat))
print(f'Test accuracy: {100*acc:.2f}%')

**Expected output:**

|            |         |
|------------|---------|
| **Train accuracy**  | 95.79% |
| **Test accuracy**  | 97.56% |

### Visualization of the training
Let's observe how the training went. The first graph plots the evolution of the loss during the training while the second graph plots the evolution of the accuracy during the training. If everything was implemented correctly, you'll notice that as training goes on, the loss decreases and the training accuracy tends to increase.

In [ ]:
# Plot the evolution of loss during training
def plot_loss(loss_list):
    fig = plt.figure(figsize=(6, 6))
    ax = fig.add_subplot(111)
    step = np.arange(1, len(loss_list)+1)
    plt.plot(step, loss_list)
    plt.title('Evolution of the loss during the training')
    plt.xlabel('iteration')
    plt.ylabel('Training loss')
    plt.show()

plot_loss(logger["loss"])

In [ ]:
# Plot the evolution of accuracy during training
def plot_acc(acc_list):
    fig = plt.figure(figsize=(6, 6))
    ax = fig.add_subplot(111)
    step = np.arange(1, len(acc_list)+1)
    plt.plot(step, acc_list)
    plt.title('Evolution of the accuracy during the training')
    plt.xlabel('iteration')
    plt.ylabel('Training accuracy')

plot_acc(logger["acc"])

### Plotting results

We also implemented functions in helpers.py which can help you visualize the decision boundary for this model, and visualize which points are correctly classified, and which aren't. This step is important in the process of developing your algorithm as it allows you to make sense of the mathematical result you obtained. 

In [ ]:
class_names = list(label_map.values())
ax_titles = ["Bill length (normalized)", "Flipper length (normalized)"]

helpers.plot_boundaries(X=X_train_penguins,
                        y=y_train_penguins, 
                        w=w, b=b,
                        output_func=logistic_output, 
                        class_names=class_names, 
                        ax_titles=ax_titles,
                        train=True)

In [ ]:
helpers.plot_boundaries(X=X_test_penguins, 
                        y=y_test_penguins, 
                        w=w, 
                        b=b, 
                        output_func=logistic_output, 
                        class_names=class_names, 
                        ax_titles=ax_titles, 
                        train=False)

If everything was implemented correctly, you'll notice that your model fits the training data with a good accuracy and also generalizes to the test data. Generalizing to new and unknown data points is the ultimate goal of any machine learning algorithm. In our case, the visualisation highlights this property.

### Training visualization

We alo implemented an interactive function which you can use to visualize the decision boundaries at different iterations. 
Notice how the decision boundary gradually improves during training?

In [ ]:
helpers.interactive_boundaries(X_train_penguins, 
                               y_train_penguins, 
                               X_test_penguins, 
                               y_test_penguins, 
                               logger["w"], 
                               logger["b"], 
                               logistic_output, 
                               class_names, 
                               ax_titles, 
                               total_steps=50)

Good job on completing the first part of this exercise! In the next parts, we'll implement a slightly modified version of logistic regression and train it on a different dataset. The functions implemented so far will prove to be very useful later on, so make sure that they are correct!

# Part 2: Regularization

The second part of this exercise consists of adding regularization to improve logistic regression's performance on a higher dimensional dataset.

## 2.1. Connectionist Bench Dataset

In this part, you'll use the Connectionist Bench Dataset, where the task is to train a model to discriminate between sonar signals bounced off a mine (metal cylinder) and those bounced off a roughly cylindrical rock. Each sample is composed of 60 features and a label: "S" if it is a stone and "M" if it is Metal (mine).

In [ ]:
sonar = pd.read_csv("data/sonar_2022.csv")

In [ ]:
print(f"Shape: {sonar.shape}")

sonar.head(5)

We use `helpers.preprocess_data()` to extract a training and test set from this data. The label "M" (metal) is mapped to 0 and the label "S" (stone) is mapped to 1.

In [ ]:
X_train_sonar, y_train_sonar, X_test_sonar, y_test_sonar, feature_names, label_map = helpers.preprocess_data(sonar, label="material", train_size=0.75, seed=42)

In [ ]:
label_map

You also need to normalize the data, as was done in the previous part. You can use `helpers.normalize()` to do so.

**Task: Normalize `X_train_sonar` and `X_test_sonar` using `helpers.normalize()`**

In [ ]:
# Normalize features of the training and test set using the mean and std of the training set features
# Use helpers.normalize()
### START CODE HERE ###
...
### END CODE HERE ###

In [ ]:
assert np.allclose(X_train_sonar.mean(axis=0), 0)
assert np.allclose(X_train_sonar.std(axis=0), 1)

In [ ]:
# Show shapes
print('Training set shape:')
print(f'X: {X_train_sonar.shape}, y: {y_train_sonar.shape}')

print('\nTest set shape:')
print(f'X: {X_test_sonar.shape}, y: {y_test_sonar.shape}')

Now that you've processed the data, let's check how well logistic regression performs without regularization.

In [ ]:
w, b, _ = train_logistic_regression(X_train_sonar, y_train_sonar, max_iters=1001, lr=0.5, loss_freq=100)

In [ ]:
# Train acc
y_hat = logistic_output(X_train_sonar, w, b)
acc = accuracy(y_train_sonar, classify(y_hat))
print(f'Train accuracy: {100 * acc:.2f}%')

In [ ]:
# Test acc
y_hat = logistic_output(X_test_sonar, w, b)
acc = accuracy(y_test_sonar, classify(y_hat))
print(f'Test accuracy: {100*acc:.2f}%')

## 2.2. Penalized logistic regression

### Penalized binary cross-entropy loss

Logistic regression can overfit when there are too many parameters compared to training examples, as it can find weights for which the decision boundary perfectly separates all the training examples. When such overfitting occurs, the weights are often set to large values. One way to reduce such overfitting is to prevent weights from becoming so large, which can be done using L2 regularization, which consists of changing the training objective to penalize "large" weights. The name L2 regularization comes from the fact that weights are penalized using the L2-norm.

We will call this new training objective (new loss function) the penalized binary-cross entropy loss (or PBCE), it is expressed as:

$$
\begin{align}
\text{PBCE}(\mathbf{w}, b) &= \text{BCE}(\mathbf{w}, b) + \lambda \|\mathbf{w}\|_{2}^{2} \\
&= [- \frac{1}{N}\sum^{N}_{i=1} y^{(i)} \log(\hat{y}^{(i)}) + (1-y^{(i)}) \log(1- \hat{y}^{(i)}) \hspace{0.2cm}] + \lambda\|\mathbf{w}\|_{2}^{2}
\end{align}$$

As you can see, this loss function consists of adding a penalty term to BCE, which **penalizes the weights but not the bias term**. The hyper-parameter $\lambda$ controls overfitting. The larger its value, the more the weights are penalized for being large, which makes the model less flexible.

**Task: Implement `penalized_bce_loss()`.**

**Note:** `lambda` is a reserved keyword in Python (used for lambda expressions), so no variables can be named this way. In our function, we use `lambd` as a variable name instead.

In [ ]:
def penalized_bce_loss(X: np.ndarray,  y: np.ndarray, w: np.ndarray, b: float, lambd: float) -> float:
    """ Penalized binary cross-entropy loss function
    
    Args:
        X: Dataset of shape (N, D)
        y: Labels of shape (N, ).
        w: Weights of logistic regression model of shape (D, )
        b: bias, a scalar
        lambd: regularization coefficient (named this way as lambda is a reserved keyword in python)
    
    Returns:
        float: binary cross-entropy loss.
    """
    ### START CODE HERE ###
    ...
    ### END CODE HERE ###
    return loss

In [ ]:
X = X_train_sonar[:3]
y  = y_train_sonar[:3]
w = np.ones(X.shape[1])
b = 1

loss = penalized_bce_loss(X, y, w, b, lambd=0.1)
assert(isinstance(loss, float))

### Penalized binary cross-entropy loss gradient

In order to use the penalized binary cross-entropy loss, you need its gradient. This time, we won't give you the gradient formula, you'll need to figure it out yourself!

**Task: Figure out the gradient for the penalized binary cross-entropy loss and implement `penalized_bce_gradient()`.**

**Hint:** Here are two useful resources for matrix calculus:
- http://www.matrixcalculus.org/
- https://en.wikipedia.org/wiki/Matrix_calculus

In [ ]:
def penalized_bce_gradient(X: np.ndarray,  
                           y: np.ndarray, 
                           w: np.ndarray, 
                           b: float, 
                           lambd: float) -> Tuple[np.ndarray, float]:
    """ Gradient of the penalized binary-cross entropy loss
    
    Args:
        X: Dataset of shape (N, D)
        y: Labels of shape (N, )
        w: Weights of logistic regression model of shape (D, )
        b: bias, a scalar
        lambd: regularization coefficient (named this way as lambda is a reserved keyword in python)
        
    Returns:
        dw (np.ndarray) gradient of the loss with respect to w of shape (D, )
        db (float) gradient of the loss with respect to b, a scalar
    """
    ### START CODE HERE ###
    ...
    ### END CODE HERE ###
    
    return dw, db
    

In [ ]:
X = X_train_sonar[:3]
y  = y_train_sonar[:3]
w = np.ones(X.shape[1])
b = 1

dw, db = penalized_bce_gradient(X, y, w, b, lambd=0.1)
assert(dw.shape == w.shape)
assert(isinstance(db, float))

### Training

Now that you've implemented the loss function and its gradient, you can use it to train your model.

**Task: Implement `train_penalized_logistic_regression()`.**

**Hint:** This training function is very similar to the one implemented in Part 1.

In [ ]:
def train_penalized_logistic_regression(X: np.ndarray, 
                                        y: np.ndarray, 
                                        lambd: float, 
                                        max_iters: int = 1001, 
                                        lr: float = 0.5, 
                                        loss_freq: int = 0) -> Tuple[np.ndarray, float, dict]:
    """ Training function for binary class penalized logistic regression using gradient descent
    
    Args:
        X: Dataset of shape (N, D).
        y: Labels of shape (N, ).
        lambd: regularization coefficient (named this way as lambda is a reserved keyword in python)
        max_iters: Maximum number of iterations.
        lr: The learning rate of  the gradient step.
        loss_freq : Prints the loss every `loss_freq` iterations.
        
    Returns:
        w: weights of shape (D, )
        b: scalar
        viz_d: dict used for visualizations
    """
    
    # Initialize weights
    np.random.seed(0)
    w = np.random.normal(0, 1, size=(X.shape[1], ))
    b = 0
    
    # Initialize dict with lists to keep track of loss, accuracy, weight and bias evolution
    logger = {'loss': [], 
             'acc': [], 
            }
    
    
    for i in range(max_iters):
        # Compute loss, dw, db and update w and b 
        ### START CODE HERE ###
        ...
        ### END CODE HERE ###
        
        # Keep track of loss and accuracy values for each iteration
        logger['loss'].append(loss)
        
        y_hat = logistic_output(X, w, b)
        logger['acc'].append(accuracy(y, classify(y_hat)))
        
        if (loss_freq !=0) and i % loss_freq == 0:
            print(f'Loss at iter {i}: {loss:.5f}')
        
    if (loss_freq != 0):
        print('\nFinal loss: {:.5f}'.format(logger['loss'][-1]))
        
    return w, b, logger

Let's now the check that there are no huge mistakes in your implementation by training your model for two different values of $\lambda$ and checking the training accuracy. Given that regularization reduces overfitting, you should expect the training accuracy to decrease when $\lambda$ increases.

In [ ]:
w, b, _ = train_penalized_logistic_regression(X_train_sonar, y_train_sonar, lambd=0, max_iters=1001, lr=0.5, loss_freq=100)
y_hat = logistic_output(X_train_sonar, w, b)
acc = accuracy(y_train_sonar, classify(y_hat))
print(f'Train accuracy: {100*acc:.2f}%')

In [ ]:
w, b, _ = train_penalized_logistic_regression(X_train_sonar, y_train_sonar, lambd=0.1, max_iters=1001, lr=0.5, loss_freq=100)
y_hat = logistic_output(X_train_sonar, w, b)
acc = accuracy(y_train_sonar, classify(y_hat))
print(f'Train accuracy: {100*acc:.2f}%')

Now that you've implemented penalized logistic regression, you may wonder which value of $\lambda$ to pick. This is what we'll try to figure out in part 3, using cross-validation.

# Part 3: Cross-validation

In this class, you saw that a dataset is usually split into 3 parts: One training set, one validation set and one test set. The training set is used as training data, the validation set is used for tuning hyper-parameters and the test set is held out for final evaluation. However, by partitioning data this way, we reduce the number of samples available for training the model, and the results on the validation depend on a particular random choice for the training and validation sets. For datasets with a small amount of training examples, this can be especially problematic.


This is where cross-validation comes into play. With cross-validation, a test set will still be held out for final evaluation, but there is no need for a designated validation set. Here, you will implement a non-exhaustive cross-validation technique known as k-fold cross-validation.

## 3.1. k-Fold Cross-validation

In k-fold cross-validation, the training data is randomly partitioned into $k$ equal sized subsamples. Of the $k$ subsamples, a single subsample is used as the validation data, and the remaining $k − 1$ subsamples are used as training data. This process is repeated $k$ times, with each of the $k$ subsamples used exactly once as the validation data. The $k$ results are then averaged to produce a single estimation. 

This process is illustrated below:

<img src="images/kfold_cv.png" alt="kfold" style="width:500px"/>

We have implemented the function `k_fold_indices()` for you, which generates indices for k-fold cross-validation. You can see its implementation and an example usage in the cell below.

In [ ]:
def k_fold_indices(num_examples: int, k: int = 4) -> List[Tuple[np.ndarray, np.ndarray]]:
    """Generates indices for k-fold cross-validation

    Args:
        num_examples: Number of training examples
        k: Number of folds

    Returns:
        List of tuples containing the training indices and validation indices

    """
    indices = np.arange(num_examples)
    split_size = num_examples // k
    val_indices = [indices[k * split_size : (k + 1) * split_size] for k in range(k)]
    both_indices = [(np.delete(indices, val_ind), val_ind) for val_ind in val_indices]
    return both_indices

# Example usage
for train_index, val_index in k_fold_indices(num_examples=8, k=4):
    # Do something with the indices
    print(f"{train_index} {val_index}\n")

**Task: With the help of `k_fold_indices()` and  previously implemented functions, implement the function `cross_val_penalized_logistic_regression()` according to its documentation.**

In [ ]:
def cross_val_penalized_logistic_regression(X: np.ndarray,
                                            y: np.ndarray,
                                            lambd: float = 0,
                                            max_iters: int = 1001,
                                            lr: float = 0.5,
                                            loss_freq: int = 0,
                                            k: int = 4) -> float:
    """
    Performs k-fold cross-validation for penalized logistic regression and returns the mean validation accuracy

    Args:
        X: Dataset of shape (N, D).
        y: Labels of shape (N, ).
        lambd: regularization coefficient (named this way as lambda is a reserved keyword in python)
        max_iters: Maximum number of iterations.
        lr: The learning rate of  the gradient step.
        loss_freq : Prints the loss every `loss_freq` iterations.
        k: Number of folds

    Returns:
        Mean validation accuracy

    """
    val_accs = []
    
    # Hint: Use a for-loop to iterate over all k-fold indices
    ### START CODE HERE ###
    ...
    ### END CODE HERE ###

    return np.mean(val_accs)

In [ ]:
mean_cv_acc = cross_val_penalized_logistic_regression(X_train_sonar, y_train_sonar)
print(f"Mean CV acc for default settings: {mean_cv_acc}")

## 3.2. Finding a good regularization parameter

You'll now use cross-validation to find a good $\lambda$ for penalized logistic regression.

**Task: Find the best value of $\lambda$**

To do so:
- Suppose that `max_iters` and `lr` are set to the **default values** of `cross_val_penalized_logistic_regression()`, and that you can only modify the hyper-parameter `lambd`.
- Compute the 4-fold cross-validation accuracy for $\lambda \in \{0, 0.001, 0.01, 0.1, 1\}.$
- Set `best_cv_acc` to the best cross-validation accuracy obtained, and `best_lambda` to the $\lambda$ associated with the best cross-validation accuracy.

In [ ]:
# Only search for values in this list, do not modify  it
lambdas = [0, 0.001, 0.01, 0.1, 1]
k = 4

# cv_accs should contain the mean cross-validation accuracy for each value of lambda
cv_accs = []

### START CODE HERE ###
...
### END CODE HERE ###

for lambd, acc in zip(lambdas, cv_accs):
    print(f"Lambda: {lambd}")
    print(f"Cross-val acc: {acc}")
    print()  
    
### START CODE HERE ###
...
### END CODE HERE ###

In [ ]:
print(f"Best lambda: {best_lambda}")
print(f"Best CV acc: {best_cv_acc}")

Now that you've settled on a value for $\lambda$, you can use it to train our model using our entire training set, and find out how well it performs on the test set.

In [ ]:
w, b, _ = train_penalized_logistic_regression(X_train_sonar, y_train_sonar, lambd=best_lambda, max_iters=1001, lr=0.5, loss_freq=100)

# Test acc
print()
y_hat = logistic_output(X_test_sonar, w, b)
acc = accuracy(y_test_sonar, classify(y_hat))
print(f'Test accuracy: {100*acc:.2f}%')

Assuming that penalized logistic regression, cross-validation and the hyperparameter search were implemented correctly, your test accuracy should be slightly higher than what was obtained for standard logistic regression in Part 2.1. While this is a modest increase, regularization can lead to much more significant increases in accuracy when working with complex machine learning models such as neural networks.

# Part 4: Evaluation metrics

Accuracy is not the only metric which can be used to measure how well your model is doing. In fact, accuracy can sometimes be a very poor metric to judge model performance. For example, suppose that you are building a machine learning model to detect whether or not someone has a disease. This disease is quite rare, so you estimate that only 1% of the people tested actually have the disease. In such a scenario, a model that always classifies a patient as negative would have an accuracy of 99%, and yet that model would be completely useless (and potentially very harmful), as it would fail to detect any positive case.

You'll now implement other metrics, which are usually preferable over accuracy when working on class-imbalanced datasets.

In this section, always suppose that:
- a ground truth label equal to **1** corresponds to a **positive example**
- a ground truth label equal to **0** corresponds to a **negative example**

Refer to this confusion matrix when implementing metrics:

**Confusion Matrix**

|                    | Actual Positive (1)    | Actual Negative (0)    |
|--------------------|---------------------|---------------------|
| **Predicted Positive (1)** | True Positive (TP)  | False Positive (FP) |
| **Predicted Negative (0)** | False Negative (FN) | True Negative (TN)  |

## 4.1. Precision

Precision attempts to answer the following question: *What proportion of positive identifications was actually correct?*

It is defined as: $$\text{Precision} = \frac{TP}{TP + FP}$$

**Task: Implement `precision()`**

In [ ]:
def precision(labels_gt: np.ndarray, labels_pred: np.ndarray) -> float:
    """Computes precision.

    Args:
        labels_gt: labels (ground-truth) of shape (M, ).
        labels_pred: Predicted labels of shape (M, ).

    Returns:
        float: Precision, in range [0, 1].
    """
        
    ### START CODE HERE ###
    ...
    ### END CODE HERE ###

In [ ]:
# Check that output is in [0, 1] (ensures that output is not a percentage)
assert 0.0 <= precision(np.array([1, 0]), np.array([1, 1])) <= 1.0

## 4.2. Recall

Recall attempts to answer the following question: *What proportion of actual positives was identified correctly?*

It is defined as: $$\text{Recall} = \frac{TP}{TP + FN}$$

**Task: Implement `recall()`**

In [ ]:
def recall(labels_gt: np.ndarray, labels_pred: np.ndarray) -> float:
    """Computes recall.

    Args:
        labels_gt: labels (ground-truth) of shape (M, ).
        labels_pred: Predicted labels of shape (M, ).

    Returns:
        float: Recall, in range [0, 1].
    """

    ### START CODE HERE ###
    ...
    ### END CODE HERE ###
 

In [ ]:
# Check that output is in [0, 1] (ensures that output is not a percentage)
assert 0.0 <= recall(np.array([1, 0]), np.array([1, 1])) <= 1.0

## 4.3. F1-score

The F1-score (or F-measure) is the harmonic mean of precision and recall: 

$$F_1 = 2 \cdot \frac{\text{precision} \cdot \text{recall}}{\text{precision} +  \text{recall}}$$

**Task: Implement `f1_score()`**

In [ ]:
   def f1_score(labels_gt: np.ndarray, labels_pred: np.ndarray) -> float:
    """Computes the F1-score.

    Args:
        labels_gt: labels (ground-truth) of shape (M, ).
        labels_pred: Predicted labels of shape (M, ).

    Returns:
        float: F1 Score, in range [0, 1].
    """
    
    ### START CODE HERE ###
    ...
    ### END CODE HERE ###

In [ ]:
# Check that output is in [0, 1] (ensures that output is not a percentage)
assert 0.0 <= f1_score(np.array([1, 0]), np.array([1, 1])) <= 1.0

## 4.4 Balanced Accuracy

The balanced accuracy is calculated as the average of the proportion of correctly classified examples for each class, it can be a good alternative to accuracy for heavily imbalanced datasets. It is defined as: $$\text{Balanced accuracy} = \frac{TPR + TNR}{2} = \frac{\frac{TP}{TP + FN} + \frac{TN}{TN + FP}}{2}$$

**Task: Implement `balanced_accuracy()`**

In [ ]:
def balanced_accuracy(labels_gt: np.ndarray, labels_pred: np.ndarray) -> float:
    """Computes the balanced accuracy

    Args:
        labels_gt: labels (ground-truth) of shape (M, ).
        labels_pred: Predicted labels of shape (M, ).

    Returns:
        float: Balanced accuracy, in range [0, 1].
    """
    ### START CODE HERE ###
    ...
    ### END CODE HERE ###

In [ ]:
# Check that output is in [0, 1] (ensures that output is not a percentage)
assert 0.0 <= balanced_accuracy(np.array([1, 0]), np.array([1, 1])) <= 1.0

## 4.5. Model evaluation

In [ ]:
print("Performance of penalized logistic regression:")
# Acc
acc = accuracy(y_test_sonar, classify(y_hat))
print(f'Test accuracy: {100 * acc:.2f}%')
# Precision
prec = precision(y_test_sonar, classify(y_hat))
print(f'Test precision: {100 * prec:.2f}%')
# Recall
rec = recall(y_test_sonar, classify(y_hat))
print(f'Test recall: {100 * rec:.2f}%')
# F1-Score
f1 = f1_score(y_test_sonar, classify(y_hat))
print(f'Test F1-score: {100 * f1:.2f}%')
# Balanced accuracy
balanced_acc = balanced_accuracy(y_test_sonar, classify(y_hat))
print(f'Test balanced accuracy: {100 * f1:.2f}%')

When using penalized logistic regression on the sonar dataset, all these metrics should give relatively similar results. The main reason for this is that this dataset is relatively well-balanced, and a lot of the implemented metrics are mostly very useful on class-imbalanced datasets.

Whew, you reached the end of this homework! The only remaining step is to submit it, so make sure to read the next section carefully.

# Submitting your homework

### Restarting the kernel

Before submitting, make sure that there are no [hidden states](https://github.com/vita-epfl/introML-2021/blob/main/exercises/00-setup/jupyter.md#the-kernel) by restarting your kernel and running all cells. Check that the code runs without errors.

In [ ]:
# Restart your kernel and run all cells, make sure that you can reach this cell
print("Ran all cells :)")

### Renaming this notebook

Rename this notebook by preprending your SCIPER (name it **SCIPER_logistic_regression.ipynb**). To do so, right click on the notebook in the left sidebar and select **"Rename"**.

For example, if your SCIPER is 123456, then name your notebook *123456_logistic_regression.ipynb* .

### Submitting on Moodle

Finally, make sure to save your notebook (File -> Save Notebook), so that it has your most recent changes. Then:
- If you are working on a **local** environment, you can directly upload this notebook (i.e. the file you are currently working on) to Moodle.
- If you are using a **cloud-based** environment such as **EPFL Noto**, you'll first need to get a local copy by right clicking on the notebook in the sidebar and clicking on **"Download"**, then upload the downloaded notebook to Moodle.